In [118]:
import pandas as pd
import numpy as np

from datetime import datetime
import time
import re

import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver import ActionChains
from selenium.webdriver import remote

from bs4 import BeautifulSoup

import requests

The following function, `select_course()`, isn't necessary unless we want to scrape tons and tons of classes. It allows us to find the 'course-id' for any given course, but does so in a rather inneficient manner by using Selenium to manually select the course we want.

In [96]:
# def find_course_id_SELENIUM(course):
#     driver = webdriver.Chrome()
#     driver.get("https://berkeleytime.com/enrollment/")
    
#     time.sleep(2)
    
#     course_selector = driver.find_element(By.ID, "react-select-2-placeholder")
#     ActionChains(driver) \
#         .click(course_selector) \
#         .send_keys(course) \
#         .perform()
#     course_dropdown = driver.find_element(By.CLASS_NAME, "css-1nmdiq5-menu")
#     first_option = course_dropdown.find_element(By.CLASS_NAME, "css-10wo9uf-option")
#     ActionChains(driver) \
#         .click(first_option) \
#         .perform()
#     time.sleep(3)
#     course_bt_id = int(driver.find_element(By.CLASS_NAME, "col-lg-4").find_elements(By.TAG_NAME, 'input')[-1].get_attribute("value"))
#     return course_bt_id

In [228]:
course_id_data = requests.get("https://berkeleytime.com/api/enrollment/enrollment_json/").json()['courses']
# course_id_data['course'] = course_id_data['abbreviation'] + " " + str(course_id_data['course_number'])
course_id_data

[{'id': 1, 'abbreviation': 'A,RESEC', 'course_number': '201'},
 {'id': 2, 'abbreviation': 'A,RESEC', 'course_number': '202'},
 {'id': 3, 'abbreviation': 'A,RESEC', 'course_number': '210'},
 {'id': 4, 'abbreviation': 'A,RESEC', 'course_number': '211'},
 {'id': 5, 'abbreviation': 'A,RESEC', 'course_number': '212'},
 {'id': 6, 'abbreviation': 'A,RESEC', 'course_number': '213'},
 {'id': 8, 'abbreviation': 'A,RESEC', 'course_number': '219A'},
 {'id': 9, 'abbreviation': 'A,RESEC', 'course_number': '219B'},
 {'id': 13451, 'abbreviation': 'A,RESEC', 'course_number': '239'},
 {'id': 11, 'abbreviation': 'A,RESEC', 'course_number': '241'},
 {'id': 12, 'abbreviation': 'A,RESEC', 'course_number': '242'},
 {'id': 13, 'abbreviation': 'A,RESEC', 'course_number': '249'},
 {'id': 14, 'abbreviation': 'A,RESEC', 'course_number': 'C251'},
 {'id': 15, 'abbreviation': 'A,RESEC', 'course_number': 'C253'},
 {'id': 16, 'abbreviation': 'A,RESEC', 'course_number': '259'},
 {'id': 17, 'abbreviation': 'A,RESEC', 'c

Here's a much more efficient way once I found where they keep the course ids:

In [230]:
# ONLY USE ONCE (the following code downloads the data that maps courses to their ids)
# course_id_data = pd.DataFrame(requests.get("https://berkeleytime.com/api/enrollment/enrollment_json/").json()['courses'])
# course_id_data['course'] = course_id_data['abbreviation'] + " " + course_id_data['course_number']
# course_id_data.to_csv('course_ids.csv', index=False)

In [267]:
def find_course_id(course):
    ids = pd.read_csv('course_ids.csv')
    if any(ids['course'] == course.upper()):
        return ids[ids['course'] == course.upper()]['id'].to_list()[0]
    else:
        print('Error: No course ID found for: ' + course)

In [253]:
id_61a = find_course_id('COMPSCI 61A')
print(id_61a)

2321

In [269]:
print(find_course_id("data c88c")) # just making sure it's not case sensitive

21485


In [83]:
test_api_url = "https://berkeleytime.com/api/enrollment/sections/" + str(id_61a) + "/"

In [84]:
r = requests.get(test_api_url)
r.status_code

200

In [87]:
e = pd.DataFrame(r.json())
e

semester  year                                           sections
0    spring  2025  [{'section_number': '001', 'section_id': 51902...
1      fall  2024  [{'section_number': '001', 'section_id': 49730...
2    spring  2024  [{'section_number': '001', 'section_id': 47595...
3      fall  2023  [{'section_number': '001', 'section_id': 45412...
4    spring  2023  [{'section_number': '001', 'section_id': 43287...
5      fall  2022  [{'section_number': '001', 'section_id': 41152...
6    spring  2022  [{'section_number': '001', 'section_id': 39017...
7      fall  2021  [{'section_number': '001', 'section_id': 37019...
8    spring  2021  [{'section_number': '001', 'section_id': 34159...
9      fall  2020  [{'section_number': '001', 'section_id': 31586...
10   spring  2020  [{'section_number': '001', 'section_id': 29890...
11     fall  2019  [{'section_number': '001', 'section_id': 28438...
12   spring  2019  [{'section_number': '001', 'section_id': 24418...
13     fall  2018  [{'section_number': '001', 'section_id': 22818...
14   spring  2018  [{'section_number': '001', 'section_id': 20172...
15     fall  2017  [{'section_number': '001', 'section_id': 19164...
16   spring  2017  [{'section_number': '001', 'section_id': 15380...
17     fall  2016  [{'section_number': '001', 'section_id': 11284...
18   spring  2016  [{'section_number': '001', 'section_id': 56692...
19     fall  2015  [{'section_number': '001', 'section_id': 46332...
20   spring  2015  [{'section_number': '001', 'section_id': 37469...
21     fall  2014  [{'section_number': '001', 'section_id': 27339...
22   spring  2014  [{'section_number': '001', 'section_id': 23467...
23     fall  2013  [{'section_number': '001', 'section_id': 8896,...

In [110]:
metadata = requests.get("https://berkeleytime.com/api/enrollment/aggregate/2321/spring/2024/").json()
metadata.pop('data')
# del metadata['data']
metadata

{'course_id': 2321,
 'section_id': 'all',
 'title': 'COMPSCI 61A',
 'subtitle': 'The Structure and Interpretation of Computer Programs',
 'section_name': 'All Sections',
 'telebears': {'phase1_start_date': '10/16/2023-00:00:00',
  'phase1_end_date': 21,
  'phase1_start_day': 1,
  'phase2_start_date': '11/14/2023-00:00:00',
  'phase2_end_date': 84,
  'phase2_start_day': 30,
  'adj_start_date': '01/08/2024-00:00:00',
  'adj_start_day': 85,
  'semester': 'Spring 2024'},
 'enrolled_max': 1000,
 'waitlisted_max': 500,
 'enrolled_percent_max': 1.1143,
 'waitlisted_percent_max': 1.1,
 'enrolled_scale_max': 1114,
 'waitlisted_scale_max': 550}

In [217]:
test_data = requests.get("https://berkeleytime.com/api/enrollment/aggregate/2321/spring/2024/").json()['data']

In [103]:
e = pd.DataFrame(test_data)
e['test'] = 'test'
e

enrolled  waitlisted  day                 date  enrolled_max  \
0           0           0   -5  10/10/2023-00:00:00           527   
1           0           0   -4  10/11/2023-00:00:00           527   
2           0           0   -3  10/12/2023-00:00:00           527   
3           0           0   -2  10/13/2023-00:00:00           527   
4           0           0   -1  10/14/2023-00:00:00           527   
..        ...         ...  ...                  ...           ...   
139       873           0  143  03/06/2024-00:00:00          1000   
140       873           0  144  03/07/2024-00:00:00          1000   
141       873           0  145  03/08/2024-00:00:00          1000   
142       873           0  146  03/09/2024-00:00:00          1000   
143       873           0  171  04/03/2024-00:00:00          1000   

     waitlisted_max  enrolled_percent  waitlisted_percent  test  
0               500             0.000                 0.0  test  
1               500             0.000                 0.0  test  
2               500             0.000                 0.0  test  
3               500             0.000                 0.0  test  
4               500             0.000                 0.0  test  
..              ...               ...                 ...   ...  
139             500             0.873                 0.0  test  
140             500             0.873                 0.0  test  
141             500             0.873                 0.0  test  
142             500             0.873                 0.0  test  
143             500             0.873                 0.0  test  

[144 rows x 9 columns]

In [276]:
def get_offering_data_fromURL(URL):
    """
    create & return dataframe with the enrollment + waitlist data for a specific offering
    (ie. the aggregated data for a specific course during a specific semester)

    URL is formatted like: https://berkeleytime.com/api/enrollment/aggregate/[course-id]/[season]/[year]/
    example" "https://berkeleytime.com/api/enrollment/aggregate/2321/spring/2024/"
    (Note: ending a request in this format automatically returns an aggregate version of the data including all offerings during any given semester.) 
    """
    raw = requests.get(URL)
    if raw.status_code > 300:
        print("Error: page not found")
        return None
    
    # Split the data into the metadata (about the course offering itself) and the timeseries data (with enrollment & waitlist counts)
    meta_data = raw.json()
    time_data = meta_data.pop('data')

    # Create a new dataframe based on the timeseries data
    df = pd.DataFrame(time_data)

    # Add new columns featuring the metadata so that if the returned dataframe were combined with other dataframes, rows would remain distinguishable
    df['course'] = meta_data['title']
    df['course_id'] = meta_data['course_id']
    df['semester'] = meta_data['telebears']['semester'].split(' ')[0]
    df['year'] = meta_data['telebears']['semester'].split(' ')[1]

    # Set variables for the start dates of each phase
    ph1_start = datetime.strptime(meta_data['telebears']['phase1_start_date'], '%m/%d/%Y-%H:%M:%S').date()
    ph2_start = datetime.strptime(meta_data['telebears']['phase2_start_date'], '%m/%d/%Y-%H:%M:%S').date()
    adj_start = datetime.strptime(meta_data['telebears']['adj_start_date'], '%m/%d/%Y-%H:%M:%S').date()

    # Adjust date formatting (add phases, fix days, etc.)
    for i in range(len(df)):
        df.loc[i, "day"] = df.loc[i, "day"] - 1 # For some reason the 'days' column doesn't match up with what is displayed on the BT website. This fixes it.
        df.loc[i, "date"] = datetime.strptime(df.loc[i, "date"], '%m/%d/%Y-%H:%M:%S').date()
        if (df.loc[i, "date"] >= adj_start):
            df.loc[i, "phase"] = 'Adjustment'
            df.loc[i, "days_after"] = int((df.loc[i, "date"] - adj_start).days)
        elif (df.loc[i, "date"] >= ph2_start):
            df.loc[i, "phase"] = 'Phase 2'
            df.loc[i, "days_after"] = int((df.loc[i, "date"] - ph2_start).days)
        else:
            df.loc[i, "phase"] = 'Phase 1'
            df.loc[i, "days_after"] = int((df.loc[i, "date"] - ph1_start).days)

    # removing negative days
    df = df.drop(df[df['day'] < 0].index)
    df = df.reset_index().drop('index', axis=1)


    # Finally, reorder the columns bc preference
    df = df[['course', 'course_id', 'semester', 'year', 'date', 'day', 'days_after', 'phase', 'enrolled', 'enrolled_max', 'enrolled_percent', 'waitlisted', 'waitlisted_max', 'waitlisted_percent']]
    return df


In [277]:
def get_offering_data(course, semester='spring', year=2025):
    course_id = str(find_course_id(course))
    semester = str(semester.lower())
    year = str(year)
    URL = "https://berkeleytime.com/api/enrollment/aggregate/" + course_id + "/" + semester + "/" + year + "/"
    return get_offering_data_fromURL(URL)

In [272]:
e = get_offering_data_fromURL("https://berkeleytime.com/api/enrollment/aggregate/2321/spring/2022/")

In [278]:
b = get_offering_data('DATA C88c', 'Spring', '2022')
b

Error: page not found


In [145]:
ph1_start = datetime.strptime(metadata['telebears']['phase1_start_date'], '%m/%d/%Y-%H:%M:%S').date()
ph2_start = datetime.strptime(metadata['telebears']['phase2_start_date'], '%m/%d/%Y-%H:%M:%S').date()
adj_start = datetime.strptime(metadata['telebears']['adj_start_date'], '%m/%d/%Y-%H:%M:%S').date()
ph1_start

datetime.date(2023, 10, 16)

In [135]:
df3.iloc[1].loc['Age'] = 3000

C:\Users\itsjh\AppData\Local\Temp\ipykernel_22604\2506425319.py:1: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df3.iloc[1].loc['Age'] = 3000
C:\Users\itsjh\AppData\Local\Temp\ipykernel_22604\2506425319.py:1: SettingWithCopyWarning: 
A valu

In [139]:
df3

Name  Age
0   John   20
1   Mary  300
2  Peter   30

In [138]:
df3.loc[1, "Age"] = 300

In [127]:
datetime.strptime('10/10/2023-00:00:00', '%m/%d/%Y-%H:%M:%S').date()

datetime.date(2023, 10, 10)

In [128]:
print(datetime.strptime('10/10/2023-00:00:00', '%m/%d/%Y-%H:%M:%S').date())

2023-10-10


In [134]:
e['date'][0]

'10/10/2023-00:00:00'

In [279]:
find_course_id('integbi 35ac')

3912

As a final test, let me use the functions i created to get the data for INTEGBI 35AC, COMPSCI 70, MATH 53, and MATH 74 -- the classes i'm taking this semester

In [284]:
courses = ['integbi 35ac', 'compsci 70', 'math 53', 'math 74']
[display(get_offering_data(course, 'Spring', '2024')) for course in courses]

course  course_id semester  year        date  day  days_after  \
0    INTEGBI 35AC       3912   Spring  2024  2023-10-16    0         0.0   
1    INTEGBI 35AC       3912   Spring  2024  2023-10-17    1         1.0   
2    INTEGBI 35AC       3912   Spring  2024  2023-10-18    2         2.0   
3    INTEGBI 35AC       3912   Spring  2024  2023-10-19    3         3.0   
4    INTEGBI 35AC       3912   Spring  2024  2023-10-20    4         4.0   
..            ...        ...      ...   ...         ...  ...         ...   
132  INTEGBI 35AC       3912   Spring  2024  2024-03-06  142        58.0   
133  INTEGBI 35AC       3912   Spring  2024  2024-03-07  143        59.0   
134  INTEGBI 35AC       3912   Spring  2024  2024-03-08  144        60.0   
135  INTEGBI 35AC       3912   Spring  2024  2024-03-09  145        61.0   
136  INTEGBI 35AC       3912   Spring  2024  2024-04-03  170        86.0   

          phase  enrolled  enrolled_max  enrolled_percent  waitlisted  \
0       Phase 1         0           299             0.000           0   
1       Phase 1         4           299             0.013           0   
2       Phase 1         7           299             0.023           0   
3       Phase 1        12           299             0.040           0   
4       Phase 1        23           299             0.077           0   
..          ...       ...           ...               ...         ...   
132  Adjustment       314           310             1.013           0   
133  Adjustment       314           310             1.013           0   
134  Adjustment       314           310             1.013           0   
135  Adjustment       314           310             1.013           0   
136  Adjustment       314           310             1.013           0   

     waitlisted_max  waitlisted_percent  
0                25                 0.0  
1                25                 0.0  
2                25                 0.0  
3                25                 0.0  
4                25                 0.0  
..              ...                 ...  
132              25                 0.0  
133              25                 0.0  
134              25                 0.0  
135              25                 0.0  
136              25                 0.0  

[137 rows x 14 columns]

course  course_id semester  year        date  day  days_after  \
0    COMPSCI 70       2327   Spring  2024  2023-10-16    0         0.0   
1    COMPSCI 70       2327   Spring  2024  2023-10-17    1         1.0   
2    COMPSCI 70       2327   Spring  2024  2023-10-18    2         2.0   
3    COMPSCI 70       2327   Spring  2024  2023-10-19    3         3.0   
4    COMPSCI 70       2327   Spring  2024  2023-10-20    4         4.0   
..          ...        ...      ...   ...         ...  ...         ...   
133  COMPSCI 70       2327   Spring  2024  2024-03-06  142        58.0   
134  COMPSCI 70       2327   Spring  2024  2024-03-07  143        59.0   
135  COMPSCI 70       2327   Spring  2024  2024-03-08  144        60.0   
136  COMPSCI 70       2327   Spring  2024  2024-03-09  145        61.0   
137  COMPSCI 70       2327   Spring  2024  2024-04-03  170        86.0   

          phase  enrolled  enrolled_max  enrolled_percent  waitlisted  \
0       Phase 1         0           481             0.000           0   
1       Phase 1        63           481             0.131          15   
2       Phase 1        86           481             0.179           0   
3       Phase 1        94           481             0.195           0   
4       Phase 1        95           481             0.198           1   
..          ...       ...           ...               ...         ...   
133  Adjustment       726           735             0.988           0   
134  Adjustment       726           735             0.988           0   
135  Adjustment       726           735             0.988           0   
136  Adjustment       726           735             0.988           0   
137  Adjustment       726           735             0.988           0   

     waitlisted_max  waitlisted_percent  
0               400               0.000  
1               400               0.037  
2               400               0.000  
3               400               0.000  
4               400               0.003  
..              ...                 ...  
133             400               0.000  
134             400               0.000  
135             400               0.000  
136             400               0.000  
137             400               0.000  

[138 rows x 14 columns]

course  course_id semester  year        date  day  days_after  \
0    MATH 53       4468   Spring  2024  2023-10-16    0         0.0   
1    MATH 53       4468   Spring  2024  2023-10-17    1         1.0   
2    MATH 53       4468   Spring  2024  2023-10-18    2         2.0   
3    MATH 53       4468   Spring  2024  2023-10-19    3         3.0   
4    MATH 53       4468   Spring  2024  2023-10-20    4         4.0   
..       ...        ...      ...   ...         ...  ...         ...   
133  MATH 53       4468   Spring  2024  2024-03-06  142        58.0   
134  MATH 53       4468   Spring  2024  2024-03-07  143        59.0   
135  MATH 53       4468   Spring  2024  2024-03-08  144        60.0   
136  MATH 53       4468   Spring  2024  2024-03-09  145        61.0   
137  MATH 53       4468   Spring  2024  2024-04-03  170        86.0   

          phase  enrolled  enrolled_max  enrolled_percent  waitlisted  \
0       Phase 1         0           728             0.000           0   
1       Phase 1        45           728             0.062           0   
2       Phase 1        50           728             0.069           0   
3       Phase 1        53           728             0.073           0   
4       Phase 1        55           728             0.076           0   
..          ...       ...           ...               ...         ...   
133  Adjustment       750           780             0.962           0   
134  Adjustment       750           780             0.962           0   
135  Adjustment       750           780             0.962           0   
136  Adjustment       750           780             0.962           0   
137  Adjustment       749           780             0.960           0   

     waitlisted_max  waitlisted_percent  
0                83                 0.0  
1                83                 0.0  
2                83                 0.0  
3                83                 0.0  
4                83                 0.0  
..              ...                 ...  
133             103                 0.0  
134             103                 0.0  
135             103                 0.0  
136             103                 0.0  
137             103                 0.0  

[138 rows x 14 columns]

course  course_id semester  year        date  day  days_after  \
0    MATH 74       4473   Spring  2024  2023-10-16    0         0.0   
1    MATH 74       4473   Spring  2024  2023-10-17    1         1.0   
2    MATH 74       4473   Spring  2024  2023-10-18    2         2.0   
3    MATH 74       4473   Spring  2024  2023-10-19    3         3.0   
4    MATH 74       4473   Spring  2024  2023-10-20    4         4.0   
..       ...        ...      ...   ...         ...  ...         ...   
132  MATH 74       4473   Spring  2024  2024-03-06  142        58.0   
133  MATH 74       4473   Spring  2024  2024-03-07  143        59.0   
134  MATH 74       4473   Spring  2024  2024-03-08  144        60.0   
135  MATH 74       4473   Spring  2024  2024-03-09  145        61.0   
136  MATH 74       4473   Spring  2024  2024-04-03  170        86.0   

          phase  enrolled  enrolled_max  enrolled_percent  waitlisted  \
0       Phase 1         0            60             0.000           0   
1       Phase 1         2            60             0.033           0   
2       Phase 1         2            60             0.033           0   
3       Phase 1         2            60             0.033           0   
4       Phase 1         2            60             0.033           0   
..          ...       ...           ...               ...         ...   
132  Adjustment        47            60             0.783           0   
133  Adjustment        47            60             0.783           0   
134  Adjustment        47            60             0.783           0   
135  Adjustment        47            60             0.783           0   
136  Adjustment        47            60             0.783           0   

     waitlisted_max  waitlisted_percent  
0                10                 0.0  
1                10                 0.0  
2                10                 0.0  
3                10                 0.0  
4                10                 0.0  
..              ...                 ...  
132              10                 0.0  
133              10                 0.0  
134              10                 0.0  
135              10                 0.0  
136              10                 0.0  

[137 rows x 14 columns]

[None, None, None, None]

In [287]:
efs = pd.concat([get_offering_data(course, 'Spring', '2024') for course in courses])
efs

course  course_id semester  year        date  day  days_after  \
0    INTEGBI 35AC       3912   Spring  2024  2023-10-16    0         0.0   
1    INTEGBI 35AC       3912   Spring  2024  2023-10-17    1         1.0   
2    INTEGBI 35AC       3912   Spring  2024  2023-10-18    2         2.0   
3    INTEGBI 35AC       3912   Spring  2024  2023-10-19    3         3.0   
4    INTEGBI 35AC       3912   Spring  2024  2023-10-20    4         4.0   
..            ...        ...      ...   ...         ...  ...         ...   
132       MATH 74       4473   Spring  2024  2024-03-06  142        58.0   
133       MATH 74       4473   Spring  2024  2024-03-07  143        59.0   
134       MATH 74       4473   Spring  2024  2024-03-08  144        60.0   
135       MATH 74       4473   Spring  2024  2024-03-09  145        61.0   
136       MATH 74       4473   Spring  2024  2024-04-03  170        86.0   

          phase  enrolled  enrolled_max  enrolled_percent  waitlisted  \
0       Phase 1         0           299             0.000           0   
1       Phase 1         4           299             0.013           0   
2       Phase 1         7           299             0.023           0   
3       Phase 1        12           299             0.040           0   
4       Phase 1        23           299             0.077           0   
..          ...       ...           ...               ...         ...   
132  Adjustment        47            60             0.783           0   
133  Adjustment        47            60             0.783           0   
134  Adjustment        47            60             0.783           0   
135  Adjustment        47            60             0.783           0   
136  Adjustment        47            60             0.783           0   

     waitlisted_max  waitlisted_percent  
0                25                 0.0  
1                25                 0.0  
2                25                 0.0  
3                25                 0.0  
4                25                 0.0  
..              ...                 ...  
132              10                 0.0  
133              10                 0.0  
134              10                 0.0  
135              10                 0.0  
136              10                 0.0  

[550 rows x 14 columns]